In [1]:
!pip install -q captum

# Imports 

In [2]:
import pandas as pd

from tqdm.notebook import tqdm,trange

import random
import numpy as np
import os

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AdamW


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix , classification_report 

# Config Setup

In [3]:
class config:
    SEED = 12
    VAL_SIZE = 0.2
    TRAIN = True
    EVALUATE = True
    INPUT_DIR = "/kaggle/input/comp-data/"
    DENSE_DIM = 2048
    MAX_SEQ_LEN = 1024
    BATCH_SIZE = 64
    NUM_WORKERS = 4
    ACC_STEPS = 1
    EPOCHS = 10
    LR = 0.002
    FOLDS = 5
    GPU = torch.cuda.is_available()
    SAVE_DIR = ""
    MODEL_SAVE_PATH =  "model.bin"


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed = config.SEED)

# Data Loading

In [5]:
target_map = {
    "C": 0,
    "E": 1,
    "H": 2,
}


reverse_target_map = {
    0: "C",
    1: "E",
    2: "H",
}

FEATURES  = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K','L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
data = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/labels_train.csv")
data.head()

,PDB_ID,SEC_STRUCT
0,1VBK_1_A,CCEEEEEECCCCCCCCCCHHHHHHHHHHHHHHHHHHCCCCCCEEEE...
1,1QGV_1_A,CCCCCCECCCHHHHHHHHHCCCCCEEEEEEECCCCHHHHHHHHHHH...
2,1KX6_1_A,CCCCCCCCCCCCCCCCCCCCCCHHHHCCC
3,1V88_1_A,CCCCCCCCCEEEEEEECCCCCCCEEEEEEEECCEEEEECCCCCCCC...
4,1ZY8_2_K,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...


# Data Processing

In [6]:
main_df_rows = []  
 
for i in trange(len(data)):
    row = data.iloc[i]
    targets = list(row["SEC_STRUCT"])
    path = f"/kaggle/input/deep-learning-for-msc-202324/train/{row['PDB_ID']}_train.csv"
    file = pd.read_csv(path)
    inputs = file[FEATURES].copy()  
    inputs["targets"] = targets
    inputs["targets"] = inputs["targets"].map(target_map)
    inputs["PDB_ID"] = row["PDB_ID"]
    

    for _, input_row in inputs.iterrows():
        main_df_rows.append(input_row.to_dict())


main_df = pd.DataFrame(main_df_rows)

main_test_df_rows = []  
test = pd.read_csv("/kaggle/input/deep-learning-for-msc-202324/seqs_test.csv")
test = test.rename(columns = {0: "PDB_ID", 1: "SEQUENCE"})

for i in trange(len(test)):
    row = test.iloc[i]
    path = f"/kaggle/input/deep-learning-for-msc-202324/test/{row['PDB_ID']}_test.csv"
    file = pd.read_csv(path)
    inputs = file[FEATURES].copy()
    inputs["PDB_ID"] = row["PDB_ID"]
    inputs["ID"] = [f"{row['PDB_ID']}_{ids}" for ids in file["RES_NUM"]]
    
    for _, input_row in inputs.iterrows():
        main_test_df_rows.append(input_row.to_dict())

main_test_df = pd.DataFrame(main_test_df_rows)

  0%|          | 0/7150 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

#  Train-Validation Data Split

In [6]:
skf = StratifiedGroupKFold(n_splits=config.FOLDS , shuffle=True, random_state=config.SEED)
for fold, (train_idx, valid_idx) in enumerate(skf.split(main_df[FEATURES], main_df["targets"], main_df["PDB_ID"])):
    break

train_df = main_df.iloc[train_idx]
val_df = main_df.iloc[valid_idx]


print("Datapoints in Train Set = ", len(train_df))
print("Datapoints in Validation Set = ", len(val_df))

Datapoints in Train Set =  1289891
Datapoints in Validation Set =  308042


# Pytorch Data Loader

In [7]:
def pad_and_truncate(data, is_test,max_len = config.MAX_SEQ_LEN):
    current_len = len(data)
    pdb_id = data['PDB_ID'].iloc[0] if not data.empty else "Unknown"
    
    if current_len < max_len:
        pad_rows = max_len - current_len
        pad_df = pd.DataFrame(-1, index=range(pad_rows), columns=FEATURES)
        pad_df['PDB_ID'] = pdb_id
        if not is_test: 
            pad_df['targets'] = -1

        data = pd.concat([data, pad_df], ignore_index=True)
    elif current_len > max_len:
        data = data.iloc[:max_len]
    
    return data


class CustomDataset(Dataset):
    def __init__(self,df, is_test = False):
        super().__init__()

        self.df = df.reset_index(drop = True).groupby("PDB_ID")      
        self.group_keys = list(self.df.groups.keys())
        self.is_test = is_test

    def __getitem__(self, idx):
        group_key = self.group_keys[int(idx)]
        data = self.df.get_group(group_key)

        data = pad_and_truncate(data, is_test = self.is_test)
        inputs = data[FEATURES].values
        
        inputs = torch.tensor(inputs, dtype = torch.float32)
        if not self.is_test:
            targets = data["targets"].values
            targets = torch.tensor(targets, dtype = torch.long)
            return inputs, targets
        else:
            return inputs
        
    def __len__(self):
        return len(self.group_keys)


# Dataloaders 

In [8]:
train_ds = CustomDataset(train_df)
val_ds = CustomDataset(val_df)
test_ds = CustomDataset(main_test_df, is_test = True)

train_loader = DataLoader(train_ds, 
                          batch_size=config.BATCH_SIZE, 
                          shuffle=True, 
                          num_workers=config.NUM_WORKERS,
                          pin_memory=False, 
                          drop_last=True)
val_loader = DataLoader(val_ds, 
                          batch_size=config.BATCH_SIZE, 
                          shuffle=True, 
                          num_workers=config.NUM_WORKERS,
                          pin_memory=False, 
                          drop_last=False)
test_loader = DataLoader(test_ds, 
                        batch_size= config.BATCH_SIZE,
                        shuffle=False, 
                        num_workers=config.NUM_WORKERS,
                        pin_memory=False, 
                        drop_last=False)

# Model

In [9]:
class ProteinSeqUNet(nn.Module):
    def __init__(self):
        super(ProteinSeqUNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(5, 5), stride=1, padding=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=(5, 5), stride=1, padding=2)
        
        self.bottleneck = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU()
        )

        self.upconv1 = nn.ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=1, padding=2)
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=1, padding=2)
        self.upconv3 = nn.ConvTranspose2d(64, 3, kernel_size=(1, 1), stride=1)

    def forward(self, x):
        x = x.unsqueeze(1)

        # Encoder
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x3 = F.relu(self.conv3(x2))
        
        x = self.bottleneck(x3)

        x = F.relu(self.upconv1(x3)) + x2  
        x = F.relu(self.upconv2(x)) + x1  
        x = self.upconv3(x)  

        x = torch.mean(x, dim=-1)

        return x


# Training Utilities

In [10]:
def read_data(data):
    """
    Function to read data
    """
    if config.GPU:
        return (data[0].to("cuda"), data[1].to("cuda"))
    else:
        return (data[0], data[1])
    
    
def compute_accuracy(outputs, targets):
    mask = targets != -1
    _, preds = torch.max(outputs, dim=1)  
    corrects = (preds == targets) & mask  
    accuracy = corrects.sum().float() / mask.sum().float() 
    return accuracy.item()


def train(model, train_loader, val_loader, config):
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    
    loss = nn.CrossEntropyLoss(ignore_index=-1)
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=config.LR)
    #optimizer = torch.optim.Adam(model.parameters(), lr =config.LR)
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []
    y_preds, y_labels = [], []

    for epoch in range(config.EPOCHS):
        model.train()
        train_loss, train_corrects, train_samples = 0.0, 0, 0
        
        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.EPOCHS}', unit='batch') as tepoch:
            for (idx,data) in enumerate(tepoch):
                inputs, targets = read_data(data)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = loss(outputs, targets)
                loss_value.backward()
                optimizer.step()

                train_loss += loss_value.item()
                train_accuracy = compute_accuracy(outputs, targets)
                train_corrects += train_accuracy * targets[targets != -1].size(0)
                train_samples += targets[targets != -1].size(0)
                
                tepoch.set_postfix(loss=f'{(train_loss/(idx+1)):.2f}', accuracy=f'{(train_corrects / train_samples) * 100:.2f}%')

        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_accuracy = train_corrects / train_samples
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_accuracy)

        model.eval()
        val_loss, val_corrects, val_samples = 0.0, 0, 0
        with torch.no_grad():
            for data in val_loader:
                inputs, targets = read_data(data)
                outputs = model(inputs)
                loss_value = loss(outputs, targets)
                val_loss += loss_value.item()

                val_accuracy = compute_accuracy(outputs, targets)
                val_corrects += val_accuracy * targets[targets != -1].size(0)
                val_samples += targets[targets != -1].size(0)

                if epoch == config.EPOCHS - 1:
                    y_preds.extend(outputs.cpu().numpy())
                    y_labels.extend(targets.cpu().numpy())

        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_accuracy = val_corrects / val_samples
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_accuracy)

        print(f'Epoch {epoch+1}, Train Loss: {epoch_train_loss:.2f}, Train Accuracy: {epoch_train_accuracy * 100:.2f}%, '
              f'Validation Loss: {epoch_val_loss:.2f}, Validation Accuracy: {epoch_val_accuracy * 100:.2f}%')

    metrics = [train_losses, train_accuracies, val_losses, val_accuracies]
    return model, metrics, np.array(y_preds), np.array(y_labels)

In [11]:
model = ProteinSeqUNet()

if config.GPU:
    model = model.to("cuda")

if config.TRAIN:
    model, metrics, y_preds, y_labels = train(model, train_loader, val_loader, config)

Epoch 1/10:   0%|          | 0/89 [00:00<?, ?batch/s]

Epoch 1, Train Loss: 1.41, Train Accuracy: 42.22%, Validation Loss: 1.05, Validation Accuracy: 44.30%


Epoch 2/10:   0%|          | 0/89 [00:00<?, ?batch/s]

Epoch 2, Train Loss: 1.05, Train Accuracy: 44.36%, Validation Loss: 1.05, Validation Accuracy: 44.30%


Epoch 3/10:   0%|          | 0/89 [00:00<?, ?batch/s]

...

Epoch 8, Train Loss: 0.88, Train Accuracy: 57.20%, Validation Loss: 0.84, Validation Accuracy: 60.40%


Epoch 9/10:   0%|          | 0/89 [00:00<?, ?batch/s]

Epoch 9, Train Loss: 0.83, Train Accuracy: 61.60%, Validation Loss: 0.83, Validation Accuracy: 62.67%


Epoch 10/10:   0%|          | 0/89 [00:00<?, ?batch/s]

Epoch 10, Train Loss: 0.78, Train Accuracy: 65.15%, Validation Loss: 0.78, Validation Accuracy: 64.93%


In [12]:
def process_labels(y_labels, y_preds):
    y_labels = y_labels.flatten()
    y_preds = y_preds.argmax(1).flatten()
    mask = y_labels!= -1
    y_preds_unmasked = y_preds[mask]
    y_labels_unmasked = y_labels[mask]
    return y_labels_unmasked, y_preds_unmasked

y_labels, y_preds = process_labels(y_labels, y_preds)

In [13]:
from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance

In [14]:
#ig = IntegratedGradients(model)

def aggregate_predictions(inp):
    """
    Assumes model output has shape (batch_size, num_classes, sequence_length)
    and computes the mean score for each class across all sequences in the batch.
    """
    # Apply the model to the input batch
    output = model(inp)
    
    # Compute the mean score for each class across all sequences
    # This assumes the last dimension is the sequence length
    aggregated_output = output.mean(dim=-1)
    
    return aggregated_output


ig = IntegratedGradients(aggregate_predictions)

In [15]:
sample_inputs, sample_outputs = next(iter(val_loader))

sample_inputs.requires_grad_()
sample_inputs = sample_inputs[:2].to("cuda")
print(sample_inputs.shape)

torch.Size([2, 1024, 20])


In [16]:
attr, delta = ig.attribute(sample_inputs,target =1, return_convergence_delta=True)
attr = attr.detach().cpu().numpy()

In [17]:
def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
    print(title)
    for i in range(len(feature_names)):
        print(feature_names[i], ": ", '%.3f'%(importances[i]))
    x_pos = (np.arange(len(feature_names)))
    if plot:
        plt.figure(figsize=(12,6))
        plt.bar(x_pos, importances, align='center')
        plt.xticks(x_pos, feature_names, wrap=True)
        plt.xlabel(axis_title)
        plt.title(title)

In [22]:
from captum.attr import LayerGradCam, FeatureAblation, LayerActivation, LayerAttribution

In [34]:
lgc = LayerGradCam(aggregate_predictions, model.conv1)


In [35]:
gc_attr = lgc.attribute(sample_inputs, target=1)

In [36]:
from captum.attr import visualization as viz

In [37]:
gc_attr.shape

torch.Size([2, 1, 1024, 20])

In [38]:
viz.visualize_image_attr(gc_attr[0].cpu().permute(1,2,0).detach().numpy(),sign="all")

(<Figure size 600x600 with 1 Axes>, <Axes: >)

In [ ]:
plt.plot(metrics[0], label = "Train Loss")
plt.plot(metrics[2], label = "Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss Plot");
plt.show()

In [ ]:
plt.plot(metrics[1], label = "Train Accuracy")
plt.plot(metrics[3], label = "Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Accuracy Plot");
plt.show()

In [ ]:
cm_data = confusion_matrix(y_preds, y_labels)
cm = pd.DataFrame(cm_data, columns=CLASS_LABELS, index = CLASS_LABELS)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (8,5))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
print(classification_report(y_preds, y_labels))

# Test Set Submission 

In [ ]:
def infer_test(model, test_loader):
    combined_out = []
    model.eval()
    with torch.no_grad():
        for inputs in test_loader:
            if config.GPU:
                inputs = inputs.to("cuda")
            outputs = model(inputs)

            combined_out.append(outputs.argmax(1))
    return torch.concat(combined_out).cpu().numpy()

def generate_submission(main_test_df, test_preds, sample_sub_path = f"{config.INPUT_DIR}sample.csv"):
    groupby_df = main_test_df.groupby("PDB_ID")
    ids = []
    targets = []
    for (idx, key) in enumerate(list(groupby_df.groups.keys())):
        df = groupby_df.get_group(key)
        ids.extend(f"{key}_{i}" for i in range(1, 1 + len(df)  ))
        targets.extend([test_preds[idx][:len(df)].tolist()][0])

    target_dict = {idd:target for idd,target in zip(ids, targets)}
    sample = pd.read_csv(sample_sub_path)
    sample["STRUCTURE"] = sample["ID"].map(target_dict).map(reverse_target_map)
    return sample


test_preds = infer_test(model, test_loader)
submission = generate_submission(main_test_df, test_preds, "/kaggle/input/deep-learning-for-msc-202324/sample.csv")
submission.to_csv("submission.csv", index = False)
submission.head()